In [ ]:
#启动代理
from browsermobproxy import Server
#存放browsermob-proxy的路径，从github上下载
server = Server(r"C:\Users\Lenovo\Downloads\Compressed\browsermob-proxy-2.1.4\bin\browsermob-proxy.bat")
server.start()
proxy = server.create_proxy()

#启动浏览器
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

#用于网页元素加载判定
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

chrome_options = Options()
chrome_options.add_experimental_option('w3c', False)
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--proxy-server={0}'.format(proxy.proxy))#指定Chrome通过此前创建的proxy
driver = webdriver.Chrome(chrome_options=chrome_options)

import pymongo

client=pymongo.MongoClient(host='localhost',port=27017)
db=client.roadshow
collection=db.video_connection

import re
f=open(r'D:\Graduation_project\video\Release_Roadshow\company.txt','r')
li=f.readlines()
prog_1=re.compile(r'[(](.*?)[)]')
prog_2=re.compile('https://rs.p5w.net/html/[0-9]+?\.shtml')
prog_3=re.compile('https://[_/a-zA-Z0-9\-\.]+?\.m3u8')
prog_4=re.compile('https://[_/a-zA-Z0-9\-\.]+?\.mp4')

import time
import subprocess
for line in li:
    company_number=prog_1.findall(line)[0]
    print(company_number)
    company_url=prog_2.findall(line)[0]
    print(company_url)
    
    company_count=collection.find({'company_number':company_number})
    url_count=collection.find({'company_number':company_url})
    
    is_insert=False
    if company_count.count()==0:
        video_info={
                'company_number':company_number,
                'video_type':"Release_Roadshow",
                'url':company_url,
        }
#         result=collection.insert(video_info)
        is_insert=True
    else:
        if url_count.count()>=1:
            print(company_number+"视频重复")
            continue
        else:
            video_info={
                'company_number':company_number,
                'video_type':"Release_Roadshow",
                'url':company_url,
            }
#             result=collection.insert(video_info)
            is_insert=True
#             company_number=company_number+"-"+str(company_count.count())

    proxy.new_har("test")
    driver.get(company_url)
    wait=WebDriverWait(driver,10)
    try:
        play=wait.until(EC.element_to_be_clickable((By.CLASS_NAME,'play')))
#         play=wait.until(EC.element_to_be_clickable((By.XPATH,'/html/body/div[4]/div/div[2]/div[1]/div/a/span')))
    except Exception:
        continue
    play.click()
    time.sleep(10)#等待页面加载完整，时间太短可能收不到包含m3u8文件信息的数据
    #proxy.wait_for_traffic_to_stop(1, 60) 
    result = proxy.har
    try:
        m3u8=prog_3.findall(str(result))[-1]#爬取的m3u8有多层，选最里面的那层最有可能包含ts流信息
    except Exception:
        m3u8=prog_4.findall(str(result))[-1]
    print(m3u8)
    if is_insert==True:
        result=collection.insert(video_info)
    print('\n')
#     #Release_Roadshow
#     out = subprocess.Popen(r'C:\Users\Lenovo\Downloads\Compressed\N_m3u8DL-CLI_v2.9.9_with_ffmpeg_and_SimpleG\N_m3u8DL-CLI_v2.9.9.exe '+
#                   m3u8+' --enableDelAfterDone'+' --workDir '+r'D:\Graduation_project\video\Release_Roadshow'+
#                   ' --saveName '+company_number, shell = False)
        
    #Listing_Ceremony
    out = subprocess.Popen(r'C:\Users\Lenovo\Downloads\Compressed\N_m3u8DL-CLI_v2.9.9_with_ffmpeg_and_SimpleG\N_m3u8DL-CLI_v2.9.9.exe '+
                  m3u8+' --enableDelAfterDone'+' --workDir '+r'D:\Graduation_project\video\Release_Roadshow'+
                  ' --saveName '+company_number+'-'+company_url, shell = False)
    
server.stop()
driver.quit()
    


    

301093
https://rs.p5w.net/html/129760.shtml


<ipython-input-5-93af7214f446>:49: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  if company_count.count()==0:
